In [1]:
import pandas as pd
%matplotlib inline
import tangos
import tangos.examples.mergers as mergers

import numpy as np
import h5py
import pickle
import matplotlib.pyplot as plt
h = 0.6776942783267969

from tangos_halo_module.halo_properties import ID_to_sim_halo_snap, get_main_progenitor_branch, impact_parameter, infall_final_n_particles, infall_final_coordinates, apocentric_distance, disruption_time, accretion_time, orbit_interpolation, infall_velocity, quenching_time, max_sSFR_time, max_mass_time 
from tangos_halo_module.path import get_file_path, get_halo_snap_num
from tangos_halo_module.halos import ID_to_tangos_halo, get_survivors, get_zombies, get_host, get_survivor_IDs, get_zombie_IDs, blockPrint, enablePrint

In [5]:
halo=ID_to_tangos_halo(ID=148208818, resolution=1000)
get_main_progenitor_branch(tangos_halo=halo, simulation='h148', resolution=1000)

(array([<Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000139/halo_25' | NDM=7936 Nstar=10 Ngas=2064>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000188/halo_29' | NDM=17080 Nstar=70 Ngas=4443>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000225/halo_30' | NDM=27025 Nstar=155 Ngas=6511>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000275/halo_34' | NDM=42312 Nstar=382 Ngas=9038>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000347/halo_28' | NDM=73673 Nstar=893 Ngas=15010>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000384/halo_21' | NDM=113654 Nstar=1361 Ngas=23831>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000456/halo_27' | NDM=119485 Nstar=2134 Ngas=22987>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000512/halo_30' | NDM=115523 Nstar=2426 Ngas=23500>,
        <Halo 'snapshots/h148.cosmo50PLK.3072g3HbwK1BH.000640/halo_31' | NDM=119741 Nstar=3142 Ngas=26568>,
        <Halo 'snapshots/h148.cosmo50PLK.3072

In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
def get_dataframe(ID, sim=0, status=0, resolution=1000):
    if ID != 0:
        sim, status, halo_id, snap_num = ID_to_sim_halo_snap(ID=ID)
    sat = ID_to_tangos_halo(ID=ID, resolution=resolution)

    infall_mass=[]; time_infall=[]; infall_mass_ratio=[]; infall_ID=[]; time_quench=[]; max_sSFR=[]; time_max_sSFR=[]
    time_disruption=[]; infall_relative_velocity=[]; infall_radial_velocity=[]; infall_tangential_velocity=[]
    max_stellarmass=[]; time_max_stellarmass=[]
    time_closest_approch=[]; distance_closest_approach_hostRvir=[]; distance_closest_approach_pkpc=[]
    infall_x=[]; infall_y=[]; infall_z=[]; infall_distance=[]; infall_phi_z=[]; infall_z_away=[]
    final_x=[]; final_y=[]; final_z=[]; final_distance=[]; final_phi_z=[]; final_z_away=[]
    infall_n_dm=[]; infall_n_star=[]; infall_n_gas=[]; final_n_dm=[]; final_n_star=[]; final_n_gas=[]

    impact_parameter_kpc=[]; impact_parameter_hostRvir=[]; time_impact=[]
    
    satinfall_time, satinfall_mass, satinfall_mass_ratio, satinfall_id, z = accretion_time(ID=ID, simulation=sim, status=status, 
                                                                                           tangos_halo=sat, halo_id=0, 
                                                                                           snap_num=0, category='stars', 
                                                                                           resolution=resolution)
    infall_mass.append(satinfall_mass)
    time_infall.append(satinfall_time)
    infall_mass_ratio.append(satinfall_mass_ratio)
    infall_ID.append(satinfall_id)

    satquenching_time, x, y = quenching_time(ID=ID, simulation=sim, status=status, 
                                       tangos_halo=sat, halo_id=0, 
                                       snap_num=0, threshold=1e-11, resolution=resolution)
    time_quench.append(satquenching_time)

#     max_satsSFR, max_satsSFR_time = max_sSFR_time(ID=ID, simulation=sim, status=status, 
#                                                   tangos_halo=sat, halo_id=0, snap_num=0, 
#                                                   resolution=resolution)
#     max_sSFR.append(max_satsSFR)
#     time_max_sSFR.append(max_satsSFR_time)

    satdisruption_time = disruption_time(ID=ID, simulation=sim, status=status, 
                                         tangos_halo=sat, halo_id=0, snap_num=0, resolution=resolution)
    time_disruption.append(satdisruption_time)

    relvel, radvel, tanvel, x = infall_velocity(ID=ID, simulation=sim, status=status, 
                                                tangos_halo=sat, halo_id=0, snap_num=0, resolution=resolution)
    infall_relative_velocity.append(relvel)
    infall_radial_velocity.append(radvel)
    infall_tangential_velocity.append(tanvel)

    max_satM, max_satM_time = max_mass_time(ID=ID, simulation=sim, status=status, 
                                            tangos_halo=sat, halo_id=0, snap_num=0, category='stars', 
                                            resolution=resolution)
    max_stellarmass.append(max_satM)
    time_max_stellarmass.append(max_satM_time)

    time_of_closest_approach, Rvir_distance, min_lin_distance = apocentric_distance(ID=ID, simulation=sim, 
                                                                                    status=status, 
                                                                                    tangos_halo=sat, 
                                                                                    halo_id=0, snap_num=0, 
                                                                                    resolution=resolution)
    time_closest_approch.append(time_of_closest_approach)
    distance_closest_approach_hostRvir.append(Rvir_distance) 
    distance_closest_approach_pkpc.append(min_lin_distance)

    (xi, yi, zi, Ri, Phii, awayi), (xf, yf, zf, Rf, Phif, awayf) = infall_final_coordinates(ID=ID, simulation=sim, 
                                                                                            status=status, 
                                                                                            tangos_halo=sat, 
                                                                                            halo_id=0, 
                                                                                            snap_num=0, 
                                                                                            resolution=resolution)
    infall_x.append(xi)
    infall_y.append(yi)
    infall_z.append(zi) 
    infall_distance.append(Ri)
    infall_phi_z.append(Phii)
    infall_z_away.append(awayi)
    final_x.append(xf) 
    final_y.append(yf) 
    final_z.append(zf) 
    final_distance.append(Rf)
    final_phi_z.append(Phif) 
    final_z_away.append(awayf)

    (gi, si, dmi), (gf, sf, dmf) = infall_final_n_particles(ID=ID, simulation=sim, status=status, tangos_halo=sat, halo_id=0, snap_num=0, 
                                                            resolution=resolution)
    infall_n_dm.append(dmi)
    infall_n_star.append(si) 
    infall_n_gas.append(gi) 
    final_n_dm.append(dmf) 
    final_n_star.append(sf)
    final_n_gas.append(gf)
    
    d_impact, d_impact_Rvir, t_impact = impact_parameter(ID=ID, simulation=sim, status=status, tangos_halo=sat, 
                                                         halo_id=0, snap_num=0, resolution=resolution)
    impact_parameter_kpc.append(d_impact) 
    impact_parameter_hostRvir.append(d_impact_Rvir)
    time_impact.append(t_impact)

    d = {'ID': ID, 
         'Simulation': sim,
         'Status': status,
         'infall_mass': infall_mass, 
         'time_infall': time_infall,
         'infall_mass_ratio': infall_mass_ratio,
         'infall_ID': infall_ID,
         'time_quench': time_quench,
#          'max_sSFR': max_sSFR,
#          'time_max_sSFR': time_max_sSFR,
         'time_disruption': time_disruption,
         'infall_relative_velocity': infall_relative_velocity,
         'infall_radial_velocity': infall_radial_velocity,
         'infall_tangential_velocity': infall_tangential_velocity,
         'max_stellarmass': max_stellarmass,
         'time_max_stellarmass': time_max_stellarmass,
         'time_closest_approch': time_closest_approch,
         'distance_closest_approach_hostRvir': distance_closest_approach_hostRvir,
         'distance_closest_approach_pkpc': distance_closest_approach_pkpc,
         'infall_x': infall_x,
         'infall_y': infall_y,
         'infall_z': infall_z,
         'infall_distance': infall_distance,
         'infall_phi_z': infall_phi_z,
         'infall_z_away': infall_z_away,
         'final_x': final_x,
         'final_y': final_y,
         'final_z': final_z,
         'final_distance': final_distance, 
         'final_phi_z': final_phi_z, 
         'final_z_away': final_z_away,
         'infall_n_dm': infall_n_dm, 
         'infall_n_star': infall_n_star, 
         'infall_n_gas': infall_n_gas, 
         'final_n_dm': final_n_dm, 
         'final_n_star': final_n_star, 
         'final_n_gas': final_n_gas, 
         'impact_parameter_kpc': impact_parameter_kpc, 
         'impact_parameter_hostRvir': impact_parameter_hostRvir, 
         'time_impact': time_impact,}

    df = pd.DataFrame(data=d)
    return df
df = get_dataframe(ID=148409612, resolution=1000)
df

,ID,Simulation,Status,infall_mass,time_infall,infall_mass_ratio,infall_ID,time_quench,time_disruption,infall_relative_velocity,infall_radial_velocity,infall_tangential_velocity,max_stellarmass,time_max_stellarmass,time_closest_approch,distance_closest_approach_hostRvir,distance_closest_approach_pkpc,infall_x,infall_y,infall_z,infall_distance,infall_phi_z,infall_z_away,final_x,final_y,final_z,final_distance,final_phi_z,final_z_away,infall_n_dm,infall_n_star,infall_n_gas,final_n_dm,final_n_star,final_n_gas,impact_parameter_kpc,impact_parameter_hostRvir,time_impact
0,148409612,h148,Survivor,2.334786e+07,8.198,0.000431,148243210,9.491,-1,236.949764,-49.322152,231.759608,7.477611e+07,9.060013,9.060013,0.778184,166.424348,76.51242,132.289877,-70.289024,168.212095,2.001884,152.822649,-52.32994,-255.495267,114.102768,284.667694,1.158376,260.79926,504066,9808,71063,212293,21027,342,166.424348,0.778184,9.06


In [3]:
# Initialize dataframe
column_names = ['ID', 'Simulation',
                'Status', 'infall_mass', 
                'time_infall', 'infall_mass_ratio', 'infall_ID',
                'time_quench', 'max_sSFR',
                'time_max_sSFR', 'time_disruption',
                'infall_relative_velocity', 'infall_radial_velocity',
                'infall_tangential_velocity', 'max_stellarmass',
                'time_max_stellarmass', 'time_closest_approch',
                'distance_closest_approach_hostRvir', 'distance_closest_approach_pkpc',
                'infall_x', 'infall_y',
                'infall_z', 'infall_distance',
                'infall_phi_z', 'infall_z_away',
                'final_x', 'final_y',
                'final_z', 'final_distance', 
                'final_phi_z', 'final_z_away',
                'infall_n_dm', 'infall_n_star', 
                'infall_n_gas', 'final_n_dm', 
                'final_n_star', 'final_n_gas']
df = pd.DataFrame(columns = column_names)
df

,ID,Simulation,Status,infall_mass,time_infall,infall_mass_ratio,infall_ID,time_quench,max_sSFR,time_max_sSFR,time_disruption,infall_relative_velocity,infall_radial_velocity,infall_tangential_velocity,max_stellarmass,time_max_stellarmass,time_closest_approch,distance_closest_approach_hostRvir,distance_closest_approach_pkpc,infall_x,infall_y,infall_z,infall_distance,infall_phi_z,infall_z_away,final_x,final_y,final_z,final_distance,final_phi_z,final_z_away,infall_n_dm,infall_n_star,infall_n_gas,final_n_dm,final_n_star,final_n_gas


In [4]:
# for sim in ['h148', 'h229', 'h242', 'h329']:
#     print(sim)
#     for status in ['Survivor', 'Zombie']:
#         print(status)
#         if status=='Survivor':
#             IDs = get_survivor_IDs(simulation=sim, resolution=100)
#         elif status=='Zombie':
#             IDs = get_zombie_IDs(simulation=sim, resolution=100)
#         for ID in IDs:
#             print('Started ', ID)
#             sat_df = get_dataframe(ID=ID, sim=sim, status=status, resolution=100)
#             df = pd.concat([df, sat_df])
#             print('Finished ', ID)
#             df.to_csv('Data100.csv', index=True)
#             print('Saved Data100.csv')
ids1000 = np.loadtxt('ids1000.txt')
ids100 = np.loadtxt('ids100.txt')

ids100=[int(i) for i in ids100]
print(len(ids100))
for idx in ids100:
    print('Started ', idx)
    sat_df = get_dataframe(ID=idx, resolution=100)
    df = pd.concat([df, sat_df])
    print('Finished ', idx)
    df.to_csv('Data100.csv', index=False)
    print('Saved Data100.csv')

194
Started  148409610
Finished  148409610
Saved Data100.csv
Started  148409610270
Finished  148409610270
Saved Data100.csv
Started  14840961090
Finished  14840961090
Saved Data100.csv
Started  14840961101
Finished  14840961101
Saved Data100.csv
Started  148409612
Finished  148409612
Saved Data100.csv
Started  14840961292
Finished  14840961292
Saved Data100.csv
Started  148409613
Finished  148409613
Saved Data100.csv
Started  1484096189
Finished  1484096189
Saved Data100.csv
Started  14840962
Finished  14840962
Saved Data100.csv
Started  1484096217
Finished  1484096217
Saved Data100.csv
Started  1484096223
Finished  1484096223
Saved Data100.csv
Started  14840962297
Finished  14840962297
Saved Data100.csv
Started  148409623
Finished  148409623
Saved Data100.csv
Started  1484096263
Finished  1484096263
Saved Data100.csv
Started  1484096290
Finished  1484096290
Saved Data100.csv
Started  14840963
Finished  14840963
Saved Data100.csv
Started  1484096306
Finished  1484096306
Saved Data100.c

/home/pathakde/MAP2021/mergers/tangos_halo_module/halo_properties.py:883: RuntimeWarning: invalid value encountered in true_divide
  (satz - hostz)**2)**(1/2)


Finished  1480225104
Saved Data100.csv
Started  148027562
Finished  148027562
Saved Data100.csv
Started  1480275869
Finished  1480275869
Saved Data100.csv
Started  1480347728
Finished  1480347728
Saved Data100.csv
Started  14803478
Finished  14803478
Saved Data100.csv
Started  1480384109
Finished  1480384109
Saved Data100.csv
Started  1480384115
Finished  1480384115
Saved Data100.csv
Started  1480384124
Finished  1480384124
Saved Data100.csv
Started  1480384163
Finished  1480384163
Saved Data100.csv
Started  14803846
Finished  14803846
Saved Data100.csv
Started  1480384990
Finished  1480384990
Saved Data100.csv
Started  1480456173
Finished  1480456173
Saved Data100.csv
Started  1480456309
Finished  1480456309
Saved Data100.csv
Started  148045663
Finished  148045663
Saved Data100.csv
Started  1480512264
Finished  1480512264
Saved Data100.csv
Started  14805124
Finished  14805124
Saved Data100.csv
Started  148051272
Finished  148051272
Saved Data100.csv
Started  148051285
Finished  148051

/home/pathakde/MAP2021/mergers/tangos_halo_module/halo_properties.py:945: RuntimeWarning: invalid value encountered in double_scalars
  cosPhi_i = z_i/R_i
/home/pathakde/MAP2021/mergers/tangos_halo_module/halo_properties.py:955: RuntimeWarning: invalid value encountered in double_scalars
  cosPhi_f = z_f/R_f


Finished  14821761398
Saved Data100.csv
Started  14824322
Finished  14824322
Saved Data100.csv
Started  14829442
Finished  14829442
Saved Data100.csv
Started  229409610
Finished  229409610
Saved Data100.csv
Started  22940961327
Finished  22940961327
Saved Data100.csv
Started  22940961381
Finished  22940961381
Saved Data100.csv
Started  229409618
Finished  229409618
Saved Data100.csv
Started  2294096191
Finished  2294096191
Saved Data100.csv
Started  229409624
Finished  229409624
Saved Data100.csv
Started  22940963661
Finished  22940963661
Saved Data100.csv
Started  229409645
Finished  229409645
Saved Data100.csv
Started  229409646
Finished  229409646
Saved Data100.csv
Started  229409648
Finished  229409648
Saved Data100.csv
Started  229409659
Finished  229409659
Saved Data100.csv
Started  229409667
Finished  229409667
Saved Data100.csv
Started  229409689
Finished  229409689
Saved Data100.csv
Started  229409691
Finished  229409691
Saved Data100.csv
Started  229010764


/home/pathakde/MAP2021/mergers/tangos_halo_module/halo_properties.py:304: RuntimeWarning: divide by zero encountered in true_divide
  Msat_ratio_accretion = M_accretion/hostMstar[t_Gyr==t_accretion]


Finished  229010764
Saved Data100.csv
Started  229013922
Finished  229013922
Saved Data100.csv
Started  22901396
Finished  22901396
Saved Data100.csv
Started  2290192184
Finished  2290192184
Saved Data100.csv
Started  2290192210
Finished  2290192210
Saved Data100.csv
Started  229019233
Finished  229019233
Saved Data100.csv
Started  229019294
Finished  229019294
Saved Data100.csv
Started  229022512
Finished  229022512
Saved Data100.csv
Started  22902252
Finished  22902252
Saved Data100.csv
Started  229022587
Finished  229022587
Saved Data100.csv
Started  229027510
Finished  229027510
Saved Data100.csv
Started  229027535
Finished  229027535
Saved Data100.csv
Started  2290288100
Finished  2290288100
Saved Data100.csv
Started  229028818
Finished  229028818
Saved Data100.csv
Started  229028831
Finished  229028831
Saved Data100.csv
Started  229028858
Finished  229028858
Saved Data100.csv
Started  22902887
Finished  22902887
Saved Data100.csv
Started  22902888
Finished  22902888
Saved Data100

In [ ]:
zombie_IDs=[]
zombies=[]
for sim in ['h148', 'h229', 'h242', 'h329']:
    zombie_IDs = zombie_IDs + get_zombie_IDs(simulation=sim)
    zombies = zombies + get_zombies(simulation=sim)
zombies

In [3]:

get_dataframe(ID=148409612, sim='h148', status='Survivor', resolution=100)

,IDs,Simulation,Status,infall_mass,time_infall,infall_mass_ratio,time_disruption,infall_relative_velocity,infall_radial_velocity,infall_tangential_velocity,...,final_z,final_distance,final_phi_z,final_z_away,infall_n_dm,infall_n_star,infall_n_gas,final_n_dm,final_n_star,final_n_gas
0,148409612,h148,Survivor,2.334786e+07,8.198053,0.000431,-1,-1,-49.322152,231.759608,...,114.102768,284.667694,1.158376,260.79926,504066,9808,71063,212293,21027,342


In [7]:
def accretion_time(ID=0, simulation=0, status=0, tangos_halo=0, halo_id=0, snap_num=0, category='stars', resolution=1000):
    '''
    input params: 
        simulation: h148, h229, h242, h329: string
        status: 'Zombie', 'Survivor': string
        tangos_halo: 0 or a valid Tangos halo object
        halo_id: halo id: 0 or string or numeric
        snap_num: 0 or 4 digit simulation snapshot number: string
        category: 'stars' or 'dm': string
    output params:
    Accretion: First time that the satellite crosses (try) 1x (else 1.5x) the virial radius of the host
               The satellite must have some stars at infall.
        mass at accretion: mass of dwarf at accretion
        0: [units: Msol]
        time of accretion: the latest time when a dwarf crosses from outside to inside 1.5 x the virial radius of the host
        1: [units: Gyr since start of simulation]
        mass ratio of satellite at accretion: mass of dwarf at accretion / mass of host at accretion
        2: [units: None]
        mass ratio of host at accretion: mass of host at accretion / mass of host at z=0
        3: [units: None]
    '''
    import pandas as pd
    if ID != 0:
        simulation, status, halo_id, snap_num = ID_to_sim_halo_snap(ID=ID)
        halo_id=int(float(halo_id))
    d = pd.read_csv('sSFR_data100.csv')
    ids = d['ID'].to_numpy()
    IDs = d['IDs'].to_numpy()
    unique_ids = IDs[ids==ID][0] # A string of form [a b c ...]
    unique_ids = np.array(list(map(float, unique_ids[1:-1].split())))
    # Accretion threshold at d*hostRvir
    d=1
    #Host
    path = get_file_path(tangos_halo=0, simulation=simulation, status='Host', halo_id='1', snap_num='4096', resolution=resolution)
    with h5py.File(path, 'r') as fh:
        a = fh['time_|_a'][:]
        t_Gyr = fh['time_|_Gyr'][:]
        hostx = fh['Xc_|_kpc'][:]
        hosty = fh['Yc_|_kpc'][:]
        hostz = fh['Zc_|_kpc'][:]
        hostMvir = fh['Mvir_|_Msol'][:]
        hostRvir = fh['Rvir_|_kpc'][:]
        hostMstar = fh['M_star_|_Msol'][:]
    t_Gyr = np.array([round(t, 3) for t in t_Gyr])
    #Satellite
    path = get_file_path(tangos_halo=tangos_halo, simulation=simulation, status=status, halo_id=halo_id, snap_num=snap_num, resolution=resolution)
    with h5py.File(path, 'r') as f:
        satx = f['Xc_|_kpc'][:]
        saty = f['Yc_|_kpc'][:]
        satz = f['Zc_|_kpc'][:]
        satnstar = f['n_star'][:]
        satMvir = f['Mvir_|_Msol'][:]
        satMstar = f['M_star_|_Msol'][:]
        
    distance = ((hostx-satx)**2 + (hosty-saty)**2 + (hostz-satz)**2)**(1/2) #pkpc
    distance_hostRvir = distance/hostRvir
    
    dips = np.zeros(len(distance))
    t_disruption = disruption_time(simulation=simulation, status=status, 
                                   tangos_halo=tangos_halo, halo_id=halo_id, snap_num=snap_num, resolution=resolution)
    if t_disruption == 0: #this is a weird zombie, which I want to exclude
        t_accretion = -1
        M_accretion = [-1]
        Msat_ratio_accretion = [-1]
        Mhost_ratio_accretion = [-1]   
    else:
        for i, val in enumerate(distance):
            if i > 0 and distance[i]<=d*hostRvir[i] and distance[i-1]>d*hostRvir[i-1] and hostMvir[i]!=satMvir[i] and satMstar[i]>0:
                dips[i]=1
            elif i > 0 and distance[i]<=1.5*d*hostRvir[i] and distance[i-1]>1.5*d*hostRvir[i-1] and hostMvir[i]!=satMvir[i] and satMstar[i]>0:
                dips[i]=2
        if 1 in dips:
            t_accretion = min(t_Gyr[dips==1]) #Gyr since beginning of simulation
        else:
            if t_disruption == -1:
                t_accretion = min(t_Gyr[(distance_hostRvir<=d+0.1) & (hostMstar>0) & (satMstar>0) & (hostMstar!=satMstar)])
            else:
                if 2 in dips:
                    t_accretion = min(t_Gyr[dips==2]) #Gyr since beginning of simulation
                else:
                    t_accretion = disruption_time(simulation=simulation, status=status, 
                                                  tangos_halo=tangos_halo, halo_id=halo_id, snap_num=snap_num)     
        t_accretion = round(t_accretion, 3)
        id_accretion = np.array(unique_ids)[t_Gyr==t_accretion]
        if category=='stars':
            M_accretion = satMstar[t_Gyr==t_accretion]
            Msat_ratio_accretion = M_accretion/hostMstar[t_Gyr==t_accretion]
            Mhost_ratio_accretion = hostMstar[t_Gyr==t_accretion]/hostMstar[-1]
            # nstar_accretion = satnstar[t_Gyr==t_accretion]
            
        elif category=='dm':
            M_accretion = satMvir[t_Gyr==t_accretion]
            Msat_ratio_accretion = M_accretion/hostMvir[t_Gyr==t_accretion]
            Mhost_ratio_accretion = hostMvir[t_Gyr==t_accretion]/hostMvir[-1]
        else:
            raise ValueError('A very specific bad thing happened. Try category= "stars" or "dm".')
           #0           #1           #2                    #3
    return t_accretion, M_accretion[0], Msat_ratio_accretion[0], int(id_accretion[0]), Mhost_ratio_accretion[0]

for idx in ids:
    print(idx)
    print(accretion_time(ID=idx, resolution=1000))

148409610
(8.198, 23347855.507486973, 0.0004310129559477759, 148243210, 0.6133501124147298)
14840961090
(6.474, 2256.328884146804, 6.050905522185976e-08, 1481920930, 0.4222151456787471)
148409612
(8.198, 23347855.507486973, 0.0004310129559477759, 148243210, 0.6133501124147298)
148409613
(13.801, 5701916.349558171, 6.456135235303597e-05, 148409613, 1.0)
1484096189
(5.868, 100251.31459288306, 3.1310520868389137e-06, 1481740150, 0.36253635254487)
14840962
(12.939, 923527877.7898424, 0.011033519553072624, 14838402, 0.9477367423515781)
1484096217
(13.801, 5952005.869089109, 6.739305253949863e-05, 1484096220, 1.0)
1484096223
(10.767, 977594.3273147542, 1.3321605023779838e-05, 1483195110, 0.8309098303420017)
148409623
(6.474, 20315105.842546724, 0.0005447999491126519, 148192013, 0.4222151456787471)
1484096263
(13.801, 519483.8382742928, 5.881983717129243e-06, 1484096260, 1.0)
1484096290
(4.75, 2071054.045395041, 9.97223074262369e-05, 148140877, 0.23515319864028053)
14840963
(9.922, 525362158.

/home/pathakde/anaconda3/envs/conda-env-py3/lib/python3.6/site-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in true_divide


(7.444, 282757.7722176811, 9.211522710062282e-06, 229220834, 0.655111019831995)
229409624
(7.444, 2214833.663484854, 7.215359786642646e-05, 229220823, 0.655111019831995)
229409645
(8.414, 3077768.8958494975, 9.485352782198608e-05, 229249618, 0.6924912642029019)
229409646
(13.262, 286863.92185006314, 6.248370721251994e-06, 229393647, 0.9798078703168028)
229409648
(7.04, 210139.44182357317, 7.017670915411356e-06, 229208840, 0.6390669474463704)
229409667
(9.706, 1252433.2418901867, 3.377012094954253e-05, 229288027, 0.7915051604841136)
229409689
(8.414, 6768.573258930633, 2.0860014954239195e-07, 229249682, 0.6924912642029019)
229409691
(9.06, 1056620.257110442, 2.974022084756787e-05, 229268850, 0.7582397079582535)
229013922
(0.477, 1363697.0885058802, 0.5205045033859461, 229013922, 5.591459745909796e-05)
22901396
(0.477, 1205448.6975737899, 0.46010325971681176, 22901396, 5.591459745909796e-05)
2290192184
(0.642, 6049.885253707847, 0.0005481560623365139, 2290188150, 0.0002355454377155023)
2

(2.919, 8097091.237448568, 0.003860607137664184, 32908649, 0.050787865460608365)
32909748
(3.242, 16081685.224694889, 0.0058755366719652966, 32909608, 0.06627817673392761)
32911527
(3.242, 35863920.056193255, 0.013103090537241445, 32909607, 0.06627817673392761)
329124811
(3.242, 35863920.056193255, 0.013103090537241445, 32909607, 0.06627817673392761)
32912695
(2.919, 53200017.39007104, 0.02536520348320597, 32908647, 0.050787865460608365)
32913443
(4.535, 175921980.01657486, 0.02072103641501301, 32913443, 0.2055867718685589)
32923043
(7.444, 123366788.73233177, 0.005848041325871718, 32922083, 0.5108276115582046)
32924962
(8.09, 414210809.0790082, 0.01745584673734235, 32924002, 0.5746018826062411)
32928803
(8.414, 144793448.7301901, 0.005835331647284387, 32924963, 0.6008553123892294)
32930722
(8.737, 446271169.9980923, 0.01719474429073514, 32925922, 0.6284773732785444)


In [2]:

quenching_time(ID=148027562, resolution=100)

0.642

In [12]:
def ID_to_sim_halo_snap(ID=0):
    '''
    input params: 
        ID: new halo ID of the format SSSXXXXYYYY [3# sim] 
                                                 +[4# latest snap num when halo exists] 
                                                 +[1-4# sim halo ID at latest snap]
    output params: 
       simulation: corresponding simulation in format 'h329'
       status: status of halo in format 'Host', 'Survivor' or 'Zombie'
       halo_id: id of halo in format '1', '283', etc.
       snap_num: final snapshot number in format '0071', '4096', etc.
    '''
    simulation = 'h'+str(ID)[:3] #eg. h329        
    snap_num = str(str(ID)[3:7])
    halo_id = str(ID)[7:]
    
    #Status:
    if snap_num == '4096':
        if halo_id == '1':
            status='Host'
        else:
            status='Survivor'
    else:
        status='Zombie'
        
    return simulation, status, halo_id, snap_num

In [9]:
ID_to_sim_halo_snap(ID=148409612)

('h148', 'Survivor', '12', '4096')

In [15]:
ids = np.loadtxt('ids100.txt')
ids=[int(i) for i in ids]
print(len(ids))
for idx in ids:
    print(idx)
    if accretion_time(ID=idx, resolution=100)[0]<=0:
        ids.remove(idx)
print(len(ids))
np.savetxt('ids100.txt', ids)

194
148409610
148409610270
14840961090
14840961101
148409612
14840961292
148409613
1484096189
14840962
1484096217
1484096223
14840962297
148409623
1484096263
1484096290
14840963
1484096306
148409636
1484096380
148409639
14840964
1484096438
148409644
148409648
1484096563
148409662
148409669
1484096779
1484096932
1480225104
148027562
1480275869
1480347728
14803478
1480384109
1480384115
1480384124
1480384163
14803846
1480384990
1480456173
1480456309
148045663
1480512264
14805124
148051272
148051285
148051289
148064011
14807012
148070125
148070149
148070165
148089619
148097431
148102421
148102429
14810249
14811066
148115210
14812803
14812806
148153610
148153614
148192016
14821761398
14824322
14829442
229409610
22940961327
22940961381
229409618
2294096191
229409624
22940963661
229409645
229409646
229409648
229409659
229409667
229409689
229409691
229010764
229013922
22901396
2290192184
2290192210
229019233
229019294
229022512
22902252
229022587
229027510
229027535
2290288100
229028818
229028